# RNN Classification Models
This example shows the application of RNN models in river-torch with and without usage of an incremental class adaption strategy.

In [1]:
from deep_river.classification import RollingClassifier
from river import metrics, compose, preprocessing, datasets
import torch
from tqdm import tqdm

## RNN Model

In [2]:
class RnnModule(torch.nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features = n_features
        self.rnn = torch.nn.RNN(
            input_size=n_features, hidden_size=hidden_size, num_layers=1
        )
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        out, hn = self.rnn(X)  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.rnn.hidden_size)
        return self.softmax(hn)

### Classification without incremental class adapation strategy

In [3]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=RnnModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=False,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=False
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [4]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

42it [00:00, 418.46it/s]

109it [00:00, 563.04it/s]

174it [00:00, 599.16it/s]

240it [00:00, 622.22it/s]

...

20213it [00:33, 593.54it/s]

20278it [00:33, 609.72it/s]

20340it [00:34, 609.87it/s]

20400it [00:34, 597.47it/s]

Accuracy: 0.02


### Classification with incremental class adaption strategy

In [5]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=RnnModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=True
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [6]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

/Users/cedrickulbach/Documents/Projects/deep-river/deep_river/base.py:237: UserWarning: The model will not be able to adapt its output to new classes since no supported output layer was found.
  warnings.warn(


59it [00:00, 581.34it/s]

118it [00:00, 556.74it/s]

180it [00:00, 584.68it/s]

...

20271it [00:33, 641.69it/s]

20336it [00:33, 638.46it/s]

20400it [00:34, 627.39it/s]

20400it [00:34, 598.45it/s]

Accuracy: 0.02


## LSTM Model

In [7]:
class LstmModule(torch.nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features = n_features
        self.lstm = torch.nn.LSTM(
            input_size=n_features, hidden_size=hidden_size, num_layers=1
        )
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        output, (hn, cn) = self.lstm(
            X
        )  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.lstm.hidden_size)
        return self.softmax(hn)

### Classifcation without incremental class adaption strategy

In [8]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=LstmModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=False
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [9]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

53it [00:00, 525.66it/s]

119it [00:00, 599.24it/s]

186it [00:00, 631.15it/s]

253it [00:00, 642.97it/s]

...

20234it [00:31, 632.57it/s]

20302it [00:31, 644.96it/s]

20373it [00:31, 661.25it/s]

20400it [00:31, 652.42it/s]

Accuracy: 0.02


### Classifcation with incremental class adaption strategy

In [10]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=LstmModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=True
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [11]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get()}:.2f")

0it [00:00, ?it/s]

67it [00:00, 666.00it/s]

135it [00:00, 670.57it/s]

203it [00:00, 662.43it/s]

270it [00:00, 648.20it/s]

...

20298it [00:35, 513.13it/s]

20350it [00:35, 494.88it/s]

20400it [00:35, 490.83it/s]

20400it [00:35, 567.64it/s]

Accuracy: 0.05965686274509804:.2f
